# <center>Livrable Projet DATA SCIENCE</center>

!pip3 install opencv-python

## Livrable 1 - Prétraitement (denoising/sharpening…)

Le but est de traiter un ensemble de photographies afin de les rendre mieux traitables par les algorithmes de Machine Learning. Il y a deux traitements à réaliser : le débruitage, et l’affutage. Vous devrez produire un notebook Jupyter explicitant ces étapes de prétraitement, et leurs performances. Ces algorithmes s’appuieront sur des notions assez simples autour des filtres de convolution, et les appliqueront pour améliorer la qualité de l’image. Il faudra notamment décider d’un compromis entre dé-bruitage et affutage.

Le notebook devra intégrer :
<ul>
    <li>Le code de chargement du fichier.</li>
    <li>Le code du débruitage sur un sous-ensemble d’images bruitées. Le code doit être accompagné d’explications.</li>
    <li>Le code de l’affutage sur un sous-ensembles d’images floutées. Le code doit être accompagné d’explications.</li>
    <li>
        Une étude de cas explicitant les compromis entre ces deux opérations. Cette partie du livrable doit inclure le bruitage d’images et montrer la perte de détails, ou l’affutage d’images et montrer l’apparition du bruit.
    </li>
</ul>

<b>Ce livrable est à fournir pour le 18/12/2020</b>

In [6]:
!pip3 install opencv-python

In [1]:
import os
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import cv2
import threading
from queue import Queue
from multiprocessing import Pool
import time

In [2]:
def get_image(path, filename):
    return io.imread(path + filename)

def save_image(path, filename, content):
    #Check if folder exists
    if not os.path.isdir(path):
        os.makedirs(path)
    imageio.imwrite(path + filename , content)
    
def get_file_amount(path):
    return len(os.listdir(path))

In [3]:
# Creer la liste de fichier a traiter
folder = "./Dataset/Blurry/"
listing = os.listdir(folder)

# 2:25 sans thraed 
# 0:40 avec thread
# L'algo est donc 3,6 fois plus rapide

# Fonction suprimant le blurr
def remove_blur(image):
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    return cv2.filter2D(image, -1, kernel)

# Thread execution
def process_fpath(name):
    print(name)
    path = folder+name
    img = get_image(folder,name)
    deblurr_img = remove_blur(img)
    save_image("./Dataset/processed/test/", name, deblurr_img)

# boucle sur la liste de fichier
if __name__ == '__main__':
    for name in listing:
        #process_fpath(name)
        t = threading.Thread(target=process_fpath, args=(name,))
        t.start()


blurry_001.jpg
blurry_002.jpg
blurry_003.jpg
blurry_004.jpg
blurry_005.jpg
blurry_006.jpg
blurry_007.jpg
blurry_008.jpg
blurry_009.jpg
blurry_010.jpg
blurry_011.jpg
blurry_012.jpg
blurry_013.jpg
blurry_014.jpg
blurry_015.jpg
blurry_016.jpg
blurry_017.jpg
blurry_018.jpgblurry_019.jpg

blurry_020.jpg
blurry_021.jpg
blurry_022.jpg
blurry_023.jpg
blurry_024.jpg
blurry_025.jpg
blurry_026.jpg
blurry_027.jpg
blurry_028.jpg
blurry_029.jpg
blurry_030.jpg
blurry_031.jpgblurry_032.jpg
blurry_033.jpg

blurry_034.jpg
blurry_035.jpg
blurry_036.jpg
blurry_037.jpg
blurry_038.jpg
blurry_039.jpg
blurry_040.jpg
blurry_041.jpg
blurry_042.jpg
blurry_043.jpg
blurry_044.jpg
blurry_045.jpg
blurry_046.jpgblurry_047.jpg
blurry_048.jpg

blurry_049.jpg
blurry_050.jpg
blurry_051.jpg
blurry_052.jpg
blurry_053.jpg
blurry_054.jpg
blurry_055.jpg
blurry_056.jpg
blurry_057.jpg
blurry_058.jpgblurry_059.jpg

blurry_060.jpg
blurry_061.jpg
blurry_062.jpgblurry_063.jpg

blurry_064.jpg
blurry_065.jpg
blurry_066.jpg
blurry_067

## Débruitage

La capture d'un signal lumineux par un appareil photographique s'accompagne le plus souvent d'informations non désirées : le « bruit ». L'essentiel de ce « bruit » (des pixels trop clairs ou trop sombre en trop grand nombre ou de manière irrégulière, par exemple) est dû au capteur.


### Le débruitage par morceaux (par patchs)

La méthode repose sur un principe simple, remplacer la couleur d'un pixel par une moyenne des couleurs de pixels similaires. Mais les pixels les plus similaires à un pixel donné n'ont aucune raison d'être proches. Il est donc necessaire de scanner une vaste partie de l'image à la recherche de tous les pixels qui ressemblent vraiment au pixel que l'on veut débruiter.

### Pourquoi cette methode ?

Le résultat d'un tel filtrage permet d’amoindrir la perte de détails au sein de l'image, comparé aux filtres réalisant des moyennes localement tel que le filtre de Gauss ou le filtre de Wiener, le bruit généré par l'algorithme "non-local means" est plus proche du bruit blanc.


In [4]:
# Prend les nom de tout les fhier dans un dossier donner
folder = "./Dataset/Noisy/"
listing = os.listdir(folder)

# Fonction suprimant le noise
def remove_noise(image):
    return  cv2.fastNlMeansDenoisingColored(image,None,10,10,7,15) 

# Thread execution
def process_fpath(name):
    print(name)
    path = folder+name
    img = get_image(folder,name)
    denoised_img = remove_noise(img)
    save_image("./Dataset/processed/test/", name, denoised_img)

#boucle sur la liste de fichier
if __name__ == '__main__':
  for name in listing:
        #process_fpath(name)
        t = threading.Thread(target=process_fpath, args=(name,))
        t.start()



noisy_001.jpg
noisy_002.jpg
noisy_003.jpg
noisy_004.jpg
noisy_005.jpg
noisy_006.jpg
noisy_007.jpg
noisy_008.jpg
noisy_009.jpg
noisy_010.jpg
noisy_011.jpg
noisy_012.jpg
noisy_013.jpg
noisy_014.jpg
noisy_015.jpg
noisy_016.jpg
noisy_017.jpg
noisy_018.jpg
noisy_019.jpg
noisy_020.jpg
noisy_021.jpg
noisy_022.jpg
noisy_023.jpg
noisy_024.jpg
noisy_025.jpg
noisy_026.jpg
noisy_027.jpg
noisy_028.jpg
noisy_029.jpg
noisy_030.jpg
noisy_031.jpg
noisy_032.jpg
noisy_033.jpg
noisy_034.jpg
noisy_035.jpg
noisy_036.jpg
noisy_037.jpg
noisy_038.jpg
noisy_039.jpg
noisy_040.jpg
noisy_041.jpg
noisy_042.jpg
noisy_043.jpg
noisy_044.jpg
noisy_045.jpg
noisy_046.jpg
noisy_047.jpg
noisy_048.jpg
noisy_049.jpg
noisy_050.jpg
noisy_051.jpg
noisy_052.jpg
noisy_053.jpg
noisy_054.jpg
noisy_055.jpg
noisy_056.jpg
noisy_057.jpg
noisy_058.jpg
noisy_059.jpg
noisy_060.jpg
noisy_061.jpg
noisy_062.jpg
noisy_063.jpg
noisy_064.jpg
noisy_065.jpg
noisy_066.jpg
noisy_067.jpg
noisy_068.jpg
noisy_069.jpg
noisy_070.jpg
noisy_071.jpg
noisy_